In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import datetime
import math, time
import sklearn
import sklearn.preprocessing
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.layers import Dense, Bidirectional
from keras.layers import LSTM
from keras.preprocessing import sequence
import csv 
import os.path
import gc
import time

C:\Users\Olegok\Anaconda3\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
min_max_scaler = sklearn.preprocessing.MinMaxScaler()
def create_dataset(seq_len = 20,stock = "AAPL", margin = 0):
    data = pd.read_csv(stock+'.csv')
    data = data.drop(['direction_up', 'direction_down','Date', 'direction'], axis=1)
    data = data.dropna(axis=0, how='any')
    
    for key in data:
        data[key] = min_max_scaler.fit_transform(data[key].values.reshape(-1,1))
    x_data = data.as_matrix()
    x_data_seq = []
    y_data_seq = []
    for index in range(len(data) - seq_len): 
        x_data_seq.append(x_data[index: index + seq_len])
        y_data_seq.append(x_data[index + seq_len]+margin)
    x_train = np.array(x_data_seq[:math.ceil(len(data)*0.8)][:][:])
    y_train = np.array(y_data_seq[:math.ceil(len(data)*0.8)][:])
    x_test = np.array(x_data_seq[math.ceil(len(data)*0.8):][:][:])
    y_test = np.array(y_data_seq[math.ceil(len(data)*0.8):][:])
    return x_train, y_train, x_test, y_test

In [3]:
def Model(_batch_size, _n_neurons=128 , _dropout=0.5, seq_len=20, _activation='relu'):
    model = Sequential()
    model.add(LSTM(_n_neurons, batch_input_shape=(_batch_size, seq_len, 36)))
    model.add(Dropout(_dropout))
    #model.add(Flatten())
    model.add(Dense(36, activation=_activation)) #relu & signoid
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [4]:
def fit_dataset(batch_size, x_train, y_train, x_test, y_test): 
    x_train = x_train[:(x_train.shape[0]//batch_size)*batch_size]
    y_train = y_train[:(y_train.shape[0]//batch_size)*batch_size]
    x_test = x_test[:(x_test.shape[0]//batch_size)*batch_size]
    y_test = y_test[:(y_test.shape[0]//batch_size)*batch_size]
    return x_train, y_train, x_test, y_test

In [5]:
n_epoch=1
n_neurons = 350
batch = 10
pred_day = 1
stock = 'AAPL'
seq = 10
file_name = 'data/'+stock +'-'+ str(pred_day) + '.csv'
data = []
start_time = time.time()
model = Model(batch,n_neurons,0.5, seq)
x_train, y_train, x_test, y_test = create_dataset(seq,stock, pred_day-1)
x_train, y_train, x_test, y_test = fit_dataset(batch, x_train, y_train, x_test, y_test)
model.fit(x_train,
     y_train,
    batch_size=batch,
     epochs=n_epoch,
   validation_data=(x_test, y_test),
    verbose=1)
score = model.evaluate(x_test, y_test, batch_size=batch)


Train on 7510 samples, validate on 1860 samples
Epoch 1/1
1860/1860 [==============================] - 2s 1ms/step


In [8]:
test_pred =  model.predict(x_test[x_test.shape[0]-100:], batch_size=batch)

In [9]:
test_pred = min_max_scaler.inverse_transform(test_pred)

In [30]:
test_pred[:,3]

array([157.12701, 148.60535, 144.86186, 145.10426, 149.3768 , 149.51122,
       156.09413, 161.37793, 159.94005, 160.23209, 160.85876, 162.92574,
       164.72786, 167.10538, 168.69847, 170.97722, 172.38152, 170.64682,
       165.50256, 162.04768, 163.8774 , 163.20074, 163.53519, 171.90703,
       177.20306, 176.74228, 174.55536, 172.20297, 166.63333, 170.93765,
       172.38597, 170.73248, 169.56456, 169.35599, 169.99316, 169.65855,
       176.71033, 175.70148, 174.03993, 172.36609, 174.62746, 172.4376 ,
       169.7412 , 168.5837 , 169.92442, 170.03935, 162.79028, 160.42484,
       161.46954, 158.81215, 166.55795, 168.25772, 169.9612 , 172.6076 ,
       170.94304, 170.58469, 169.8471 , 171.71721, 171.89494, 167.11423,
       173.28465, 174.97163, 175.41833, 172.17685, 170.98776, 167.39386,
       164.72043, 166.10638, 166.06598, 166.97661, 166.15236, 166.13785,
       163.19196, 161.59537, 166.86768, 164.17128, 155.60397, 156.16031,
       162.6512 , 163.95189, 164.75638, 164.20609, 

In [66]:
data = pd.read_csv('AAPL.csv')
data = data.drop(['direction_up', 'direction_down', 'direction'], axis=1)
data = data[data.shape[0]-100:].reset_index()

In [67]:
data["prediction"] = pd.Series(test_pred[:,3])

In [72]:
data["prediction"] = data["prediction"].shift(1)

In [73]:
data = data.dropna(axis=0, how='any')

In [77]:
data.to_csv('out.csv', index=False)